In [24]:
import os,shutil,importlib
import ecProject as dev
importlib.reload(dev)

projectPath = os.getcwd()+'/temp/'
mp = dev.myProject(projectPath)
mp.projectConfig
# if os.path.isdir(projectPath):
#     print(projectPath)
#     shutil.rmtree(projectPath)
# myProject = dev.makeProject(projectPath)

{'Database': {'Purpose': 'Binary (float32) timeseries arrays and corresponding posix timemstamp (float64) in a subdirectory structure that partitions data by Year, siteID, and cleaning stage',
  'Structure': 'YYYY/siteID/stage',
  'Metadata': {'Data_traces': {'dtype': 'float32', 'precision': 4},
   'POSIX_timestamp': {'dtype': 'float64', 'precision': 8, 'base_unit': 's'},
   'Time_resolution': '30min'},
  'inventory': None},
 'rawSiteData': {'Purpose': 'Source files for the project, stored by with an inventory to track whether files have been dumped to the database.  Contents are organized by siteID, Type, Year, and Month, if timestamps are can be parsed from the filename or just siteID and Type if no time information available.',
  'inventory': None},
 'config_files': {'Purpose': 'Configuration files', 'inventory': None}}

In [68]:
import os,shutil,importlib
import development as dev
importlib.reload(dev)

projectPath = 'D:/temp/ArcticFlux'

for ID,tag,dIn in zip(['FIL','SCL'],
                  ['CR1000','Met_Data'],
                  ['D:\\MetStationData\\FishIsland',
                  'D:\\highfreq\SCL\\CardConvertDump'
                  ]
                  ):
    print(ID,tag)
    myProject = dev.dataDump(projectPath,
                            siteID=ID,
                            dIn=dIn,
                            fileType='TOA5',
                            searchTag = tag,
                            subPath='Met'
                            )


FIL CR1000
Searching  D:\MetStationData\FishIsland
Warning!!

Unable to parse date pattern from input file, proceeding without
Creating template:  D:\temp\ArcticFlux\rawSiteData/FIL/TOA5/Met/metaData.yml 

Edit this file before proceeding with import


Copying:[████████████████████████████████████████████████████████████] 1/1 

SCL Met_Data
Outstanding imports:  1
Searching  D:\highfreq\SCL\CardConvertDump
Creating template:  D:\temp\ArcticFlux\rawSiteData/SCL/TOA5/Met/metaData.yml 

Edit this file before proceeding with import


Copying:[████████████████████████████████████████████████████████████] 10/10 

Copying:[████████████████████████████████████████████████████████████] 31/31 

Copying:[████████████████████████████████████████████████████████████] 13/13 

